# IMPORTING ALL LIBRARIES

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


from keras.layers import Dense, SimpleRNN, Embedding
from keras.models import Sequential

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.text import Tokenizer     # used to assign a unique number for every unique word
from tensorflow.keras.preprocessing.sequence import pad_sequences   # will add extra zeros 

from keras.layers import Dropout

# UPLOADING THE FILE

In [2]:
gr=pd.read_csv(r'C:\Users\Amaljith P\Desktop\CLASS\PYTHON\FILES\greviance.csv')

# DATA PRE PROCESSING


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

gr.Product = le.fit_transform(gr.Product)
gr.Product.unique()

array([ 6,  8,  1, 18, 12,  7,  3, 19, 16, 15, 10, 17,  4,  2,  9, 14,  0,
        5, 13, 11, 20])

In [6]:
gr['x'] = (
    gr['Issue'].apply(str) + "   " +
    gr['Sub-issue'].apply(str) + "   " +
    gr['Consumer complaint narrative'].apply(str) + "   " +
    gr['Company'].apply(str)
)

In [7]:
gr = gr.iloc[:,[1,-1]]   
gr.head(1)

,Product,x
0,6,Incorrect information on your report Informa...


# SPLITTING THE DATA AND MODEL BUILDING

In [14]:
gr_x = gr.iloc[: , 1]
gr_y = gr.iloc[: , 0]

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(gr_x, gr_y, test_size=0.2) 

In [15]:
train_y = to_categorical(train_y)

In [16]:
max_num_words  = 30000    
seq_len        = 300      
embedding_size = 200 

In [17]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(gr.x)

train_x = tokenizer.texts_to_sequences(train_x)   # will convert the text to sequences of IDs
train_x = pad_sequences(train_x, maxlen = seq_len)

In [18]:
test_x = tokenizer.texts_to_sequences(test_x)  
test_x = pad_sequences(test_x, maxlen = seq_len)


In [19]:
model = Sequential()         # initialize the network
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len,
                    output_dim = embedding_size))      # HYPERPARAMETERS WHICH CAN BE CHANGED FROM ABOVE
model.add(LSTM(128))    # No. of neurons in the single hidden layer HYPERPARAMETER
model.add(Dropout(0.5))
model.add(Dense(21, activation = 'softmax'))
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
model.fit(train_x, train_y, epochs = 5, batch_size = 1024, validation_split = .02)

Epoch 1/5
3278/3278 ━━━━━━━━━━━━━━━━━━━━ 4474s 1s/step - accuracy: 0.8538 - loss: 0.5393 - val_accuracy: 0.9241 - val_loss: 0.2525
Epoch 2/5
3278/3278 ━━━━━━━━━━━━━━━━━━━━ 12080s 4s/step - accuracy: 0.9267 - loss: 0.2474 - val_accuracy: 0.9351 - val_loss: 0.2080
Epoch 3/5
3278/3278 ━━━━━━━━━━━━━━━━━━━━ 402414s 123s/step - accuracy: 0.9352 - loss: 0.2103 - val_accuracy: 0.9372 - val_loss: 0.1985
Epoch 4/5
3278/3278 ━━━━━━━━━━━━━━━━━━━━ 4759s 1s/step - accuracy: 0.9363 - loss: 0.2032 - val_accuracy: 0.9385 - val_loss: 0.1924
Epoch 5/5
3278/3278 ━━━━━━━━━━━━━━━━━━━━ 4829s 1s/step - accuracy: 0.9398 - loss: 0.1884 - val_accuracy: 0.9382 - val_loss: 0.1915


In [21]:
pred = model.predict(test_x)

pred_classes = pred.argmax(axis=1)    
pred_classes

26754/26754 ━━━━━━━━━━━━━━━━━━━━ 1119s 42ms/step


array([ 1,  7, 12, ...,  8,  1, 18], dtype=int64)

In [22]:
from sklearn.metrics import confusion_matrix, classification_report
tab=confusion_matrix(test_y, pred_classes)
print(tab)
print('-------------------------')
print(classification_report(test_y, pred_classes))
print('accuracy in decimals',tab.diagonal().sum() / tab.sum()*100)

[[ 16818    219      1     34     40      1      0     11     22      0
      14     10     52      0      0      2      0      4      1      1
       0]
 [   122  36335      0      8    378      0      1    261     57      0
     338      0    100      0      0     11      0      1      1      8
       0]
 [     6      4   6002     11     16      1      0     24     38      0
       0      0     32      0     69     36      0      0     24     86
       0]
 [    40     15      3  17623   1777      0    423    128     51      0
       4      1     24      3      1      4      0      3      1      0
       0]
 [    20    310      2    126  37722      0      2   2597    128      0
     129      0     46      0      0     34      0     32      4     18
       0]
 [     0      0      0      2      3  27694      1    298      4      0
       0      0      0      0      0      0      0      0      1      0
       0]
 [     0      1      0     14      7      1  10057  34167     10      0
    